In [7]:
import requests
import json

url_base = "http://localhost:8000"

In [8]:
# TESTE DE STRESS SIMPLES
import time

def stress_test_batch():
    """
    Teste de stress simples para o endpoint batch:
    - Envia 5 comparações (máximo permitido)
    - Mede tempos de resposta
    - Verifica se todas as comparações foram processadas
    - Compara performance com execuções individuais
    """
    print("🚀 === INICIANDO TESTE DE STRESS BATCH ===\n")
    
    # Dados para o teste de stress (5 comparações - máximo permitido)
    stress_data = {
        "comparisons": [
            {
                "input": "Qual a capital do Brasil?",
                "response_a": "A capital do Brasil é Brasília, localizada no Distrito Federal.",
                "response_b": "Brasília é a capital do Brasil desde 1960.",
                "model_a_name": "claude-4-sonnet",
                "model_b_name": "gemini-2.5-pro"
            },
            {
                "input": "Qual é o maior planeta do sistema solar?",
                "response_a": "O maior planeta do sistema solar é Júpiter.",
                "response_b": "Júpiter é o maior planeta, com massa maior que todos os outros planetas combinados.",
                "model_a_name": "llama-4-maverick",
                "model_b_name": "claude-4-sonnet"
            },
            {
                "input": "Quem foi Albert Einstein?",
                "response_a": "Albert Einstein foi um físico teórico alemão.",
                "response_b": "Einstein foi um físico que desenvolveu a teoria da relatividade.",
                "model_a_name": "gemini-2.5-pro", 
                "model_b_name": "llama-4-maverick"
            },
            {
                "input": "O que é inteligência artificial?",
                "response_a": "Inteligência artificial é a capacidade de máquinas realizarem tarefas que requerem inteligência humana.",
                "response_b": "IA é um campo da ciência da computação focado em criar sistemas inteligentes.",
                "model_a_name": "claude-4-sonnet",
                "model_b_name": "gemini-2.5-pro"
            },
            {
                "input": "Como funciona o aquecimento global?",
                "response_a": "O aquecimento global ocorre devido ao aumento de gases de efeito estufa na atmosfera.",
                "response_b": "É o aumento da temperatura média da Terra causado pela atividade humana.",
                "model_a_name": "llama-4-maverick",
                "model_b_name": "claude-4-sonnet"
            }
        ]
    }
    
    # Fazer 3 execuções para verificar consistência
    results = []
    total_time = 0
    
    for i in range(1, 4):
        print(f"📊 Execução {i}/3 - {len(stress_data['comparisons'])} comparações...")
        
        start_time = time.time()
        response = safe_request("post", url_base + endpoint, json=stress_data, timeout=120)  # timeout maior
        execution_time = time.time() - start_time
        total_time += execution_time
        
        if response and response.status_code == 200:
            data = safe_json_response(response)
            if data:
                results.append({
                    'execution': i,
                    'status': 'success',
                    'api_time': data['execution_time'],
                    'total_time': execution_time,
                    'comparisons': data['total_comparisons'],
                    'successful': data['successful'],
                    'errors': data['errors'],
                    'model_a_wins': data['model_a_wins'],
                    'model_b_wins': data['model_b_wins'],
                    'ties': data['ties']
                })
                print(f"   ✅ Sucesso - {data['successful']}/{data['total_comparisons']} comparações")
                print(f"   ⏱️  Tempo API: {data['execution_time']:.2f}s | Total: {execution_time:.2f}s")
                print(f"   🏆 A:{data['model_a_wins']} B:{data['model_b_wins']} Empates:{data['ties']} Erros:{data['errors']}\n")
            else:
                results.append({'execution': i, 'status': 'json_error'})
                print(f"   ❌ Erro ao processar JSON\n")
        else:
            results.append({'execution': i, 'status': 'request_error'})
            print(f"   ❌ Erro na requisição - Status: {response.status_code if response else 'None'}\n")
    
    # Análise dos resultados
    print("📈 === ANÁLISE DE PERFORMANCE ===")
    successful_runs = [r for r in results if r['status'] == 'success']
    
    if successful_runs:
        avg_api_time = sum(r['api_time'] for r in successful_runs) / len(successful_runs)
        avg_total_time = sum(r['total_time'] for r in successful_runs) / len(successful_runs)
        total_comparisons_processed = sum(r['successful'] for r in successful_runs)
        total_errors = sum(r['errors'] for r in successful_runs)
        
        print(f"✅ Execuções bem-sucedidas: {len(successful_runs)}/3")
        print(f"⏱️  Tempo médio API: {avg_api_time:.2f}s")
        print(f"⏱️  Tempo médio total: {avg_total_time:.2f}s")
        print(f"🚀 Comparações/segundo: {5/avg_api_time:.2f}")
        print(f"📊 Total de comparações processadas: {total_comparisons_processed}")
        print(f"❌ Total de erros: {total_errors}")
        
        # Comparação com processamento individual estimado
        estimated_individual_time = avg_api_time / 5 * 5  # tempo por comparação * 5 (sequencial)
        speedup = estimated_individual_time / avg_api_time if avg_api_time > 0 else 0
        print(f"🏃 Speedup estimado vs. individual: {speedup:.1f}x")
        
        # Estatísticas agregadas
        total_a_wins = sum(r['model_a_wins'] for r in successful_runs)
        total_b_wins = sum(r['model_b_wins'] for r in successful_runs)
        total_ties = sum(r['ties'] for r in successful_runs)
        
        print(f"\n🏆 === ESTATÍSTICAS AGREGADAS (todas as execuções) ===")
        print(f"📊 Modelo A total: {total_a_wins} vitórias")
        print(f"📊 Modelo B total: {total_b_wins} vitórias") 
        print(f"📊 Empates total: {total_ties}")
        
        if total_a_wins + total_b_wins + total_ties > 0:
            print(f"🎯 Taxa de decisão: {((total_a_wins + total_b_wins) / (total_a_wins + total_b_wins + total_ties) * 100):.1f}%")
        
    else:
        print("❌ Todas as execuções falharam")
    
    print(f"\n⏱️  Tempo total do teste: {total_time:.2f}s")
    print("🏁 === TESTE DE STRESS CONCLUÍDO ===")
    
    return results

# Executar o teste de stress
stress_results = stress_test_batch()

🚀 === INICIANDO TESTE DE STRESS BATCH ===

📊 Execução 1/3 - 5 comparações...
   ✅ Sucesso - 5/5 comparações
   ⏱️  Tempo API: 2.25s | Total: 2.26s
   🏆 A:2 B:0 Empates:3 Erros:0

📊 Execução 2/3 - 5 comparações...
   ✅ Sucesso - 5/5 comparações
   ⏱️  Tempo API: 1.89s | Total: 1.89s
   🏆 A:2 B:0 Empates:3 Erros:0

📊 Execução 3/3 - 5 comparações...
   ✅ Sucesso - 5/5 comparações
   ⏱️  Tempo API: 1.56s | Total: 1.56s
   🏆 A:2 B:0 Empates:3 Erros:0

📈 === ANÁLISE DE PERFORMANCE ===
✅ Execuções bem-sucedidas: 3/3
⏱️  Tempo médio API: 1.90s
⏱️  Tempo médio total: 1.90s
🚀 Comparações/segundo: 2.63
📊 Total de comparações processadas: 15
❌ Total de erros: 0
🏃 Speedup estimado vs. individual: 1.0x

🏆 === ESTATÍSTICAS AGREGADAS (todas as execuções) ===
📊 Modelo A total: 6 vitórias
📊 Modelo B total: 0 vitórias
📊 Empates total: 9
🎯 Taxa de decisão: 40.0%

⏱️  Tempo total do teste: 5.71s
🏁 === TESTE DE STRESS CONCLUÍDO ===


In [9]:
endpoint = "/"

response = requests.get(url_base + endpoint)
print(response.json())

{'message': 'LLM as Judge Study API', 'version': '0.1.0', 'endpoints': {'compare': '/api/v1/compare', 'models': '/api/v1/models', 'health': '/api/v1/health', 'docs': '/docs'}}


In [10]:
endpoint = "/api/v1/compare"

data = {
    "input": "Qual a capital do Brasil?",
    "response_a": "A capital do Brasil é Brasília, localizada no Distrito Federal.",
    "response_b": "Brasília é a capital do Brasil desde 1960."
}

response = requests.post(url_base + endpoint, json=data, timeout=10)
print(response.json())

{'input': 'Qual a capital do Brasil?', 'response_a': 'A capital do Brasil é Brasília, localizada no Distrito Federal.', 'response_b': 'Brasília é a capital do Brasil desde 1960.', 'better_response': 'Empate', 'judge_reasoning': None, 'model_a_name': None, 'model_b_name': None, 'timestamp': '2025-08-24T19:01:24.266315Z', 'execution_time': 1.162778377532959}


In [11]:
endpoint = "/api/v1/models"

response = requests.get(url_base + endpoint, timeout=10)
print(response.json())

{'available_models': ['llama-4-maverick', 'claude-4-sonnet', 'google-gemini-2.5-pro'], 'total_count': 3}


In [12]:
endpoint = "/api/v1/health"

response = requests.get(url_base + endpoint, timeout=10)
print(response.json())

{'status': 'healthy', 'timestamp': '2025-08-24T16:01:24.363909', 'service': 'LLM as Judge Study API', 'version': '0.1.0'}


In [13]:
# Definir configurações base
url_base = "http://localhost:8000"
endpoint = "/api/v1/compare/batch"

# Dados de teste para o endpoint batch
data_valid = {
    "comparisons": [
        {
            "input": "Qual a capital do Brasil?",
            "response_a": "A capital do Brasil é Brasília.",
            "response_b": "Brasília é a capital do Brasil desde 1960.",
            "model_a_name": "claude-4-sonnet",
            "model_b_name": "gemini-2.5-pro"
        },
        {
            "input": "Qual é o maior planeta do sistema solar?",
            "response_a": "O maior planeta do sistema solar é Júpiter.",
            "response_b": "Júpiter é o maior planeta, com massa maior que todos os outros planetas combinados.",
            "model_a_name": "llama-4-maverick",
            "model_b_name": "claude-4-sonnet"
        }
    ]
}

data_invalid = {
    "comparisons": [
        {"input": "Pergunta " + str(i), "response_a": f"Resposta A{i}", "response_b": f"Resposta B{i}"}
        for i in range(1, 7)  # 6 comparações - excede limite de 5
    ]
}

data_single = {
    "comparisons": [
        {
            "input": "Qual a capital do Brasil?",
            "response_a": "A capital do Brasil é Brasília.",
            "response_b": "Brasília é a capital do Brasil desde 1960."
        }
    ]
}

data_empty = {"comparisons": []}

data_no_payload = {}

def safe_request(method, url, **kwargs):
    """Faz request com tratamento seguro de erros"""
    try:
        response = getattr(requests, method)(url, **kwargs)
        return response
    except requests.RequestException as e:
        print(f"Erro de conexão: {e}")
        return None

def safe_json_response(response):
    """Extrai JSON com tratamento seguro de erros"""
    if response is None:
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Erro ao decodificar JSON. Status: {response.status_code}")
        print(f"Raw response: {response.text[:200]}")
        return {"error": "Invalid JSON response", "status_code": response.status_code}

def print_response_info(response, test_name):
    """Imprime informações da resposta de forma padronizada"""
    print(f"\n=== {test_name} ===")
    
    if response is None:
        print("❌ Falha na conexão")
        return
    
    print(f"Status: {response.status_code}")
    
    # Verificar se resposta tem conteúdo
    if not response.text.strip():
        print("❌ Resposta vazia")
        return
        
    response_data = safe_json_response(response)
    
    if response_data:
        print("Response:", json.dumps(response_data, indent=2, ensure_ascii=False))
        
        # Destacar estatísticas de performance apenas para respostas 200
        if response.status_code == 200 and 'model_a_wins' in response_data:
            print("\n🏆 === ESTATÍSTICAS DE PERFORMANCE ===")
            print(f"📊 Modelo A (vitórias): {response_data['model_a_wins']}")
            print(f"📊 Modelo B (vitórias): {response_data['model_b_wins']}")
            print(f"📊 Empates: {response_data['ties']}")
            print(f"📊 Erros: {response_data['errors']}")
            print(f"🥇 Melhor modelo geral: {response_data['best_model']}")
            print(f"⏱️ Tempo de execução: {response_data['execution_time']:.2f}s")
    else:
        print("❌ Falha ao processar resposta JSON")

# Executar testes
print("🚀 Iniciando testes do endpoint batch...")

# TESTE 1: Request válido
response = safe_request("post", url_base + endpoint, json=data_valid, timeout=30)
print_response_info(response, "TESTE 1: Request válido (2 comparações)")

# TESTE 2: Request inválido (muitas comparações)
response = safe_request("post", url_base + endpoint, json=data_invalid, timeout=30)
print_response_info(response, "TESTE 2: Request inválido (6 comparações - limite é 5)")

# TESTE 3: Request com apenas 1 comparação (deve sugerir endpoint individual)
response = safe_request("post", url_base + endpoint, json=data_single, timeout=30)
print_response_info(response, "TESTE 3: Apenas 1 comparação (deve sugerir endpoint individual)")

# TESTE 4: Lista vazia
response = safe_request("post", url_base + endpoint, json=data_empty, timeout=30)
print_response_info(response, "TESTE 4: Lista vazia (deve dar erro - mínimo 2)")

# TESTE 5: Sem payload
response = safe_request("post", url_base + endpoint, json=data_no_payload, timeout=30)
print_response_info(response, "TESTE 5: Sem payload (deve dar erro - campo obrigatório)")

# TESTE 6: Sem JSON
response = safe_request("post", url_base + endpoint, timeout=30)
print_response_info(response, "TESTE 6: Sem JSON (deve dar erro - body vazio)")

print("\n✅ Testes concluídos!")

🚀 Iniciando testes do endpoint batch...

=== TESTE 1: Request válido (2 comparações) ===
Status: 200
Response: {
  "results": [
    {
      "id": "627f8b17-159b-47b8-9bb2-dd4da88eb82b",
      "input": "Qual a capital do Brasil?",
      "response_a": "A capital do Brasil é Brasília.",
      "response_b": "Brasília é a capital do Brasil desde 1960.",
      "model_a_name": "claude-4-sonnet",
      "model_b_name": "gemini-2.5-pro",
      "better_response": "Empate",
      "judge_reasoning": null
    },
    {
      "id": "962e1a85-9d73-4a98-9753-f0427f32ce33",
      "input": "Qual é o maior planeta do sistema solar?",
      "response_a": "O maior planeta do sistema solar é Júpiter.",
      "response_b": "Júpiter é o maior planeta, com massa maior que todos os outros planetas combinados.",
      "model_a_name": "llama-4-maverick",
      "model_b_name": "claude-4-sonnet",
      "better_response": "Empate",
      "judge_reasoning": null
    }
  ],
  "total_comparisons": 2,
  "successful": 2,
 